# 프로젝트-얼굴을 인식하여 캐릭터 씌우기

## FaceDetection vs Face Recognition
1.FaceDetection :얼굴만 디텍션  
2.FaceRecognition : 누군인지 구분  
https://mediapipe.dev/  
https://viso.ai/deep-learning/face-detection-overview/

In [1]:
import cv2
import mediapipe as mp

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈을 사용 
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 Drawing_Utils 모듈을 사용 

# 영상 소스 선택 0 이면 WebCam임 
cap = cv2.VideoCapture(0) # Webcam 
# cap = cv2.VideoCapture('face_video.mp4') # Video
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.9) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 디텍션 된 얼굴에 무언가를 그려줌
        if results.detections:
            # 6개 특징 : 오른쪽눈, 왼쪽눈, 코끝부분, 입중심, 오른쪽 귀, 왼쪽귀 
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)
                # print(detection)
                
                # 특정 위치 가져오기 
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]
                nose_tip = keypoints[2]
                
                h, w,_ = image.shape
                right_eye = (int(right_eye.x * w), int(right_eye.y * h))
                left_eye = (int(left_eye.x * w), int(left_eye.y * h))
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # 양눈에 동그라미 그리기
                cv2.circle(image,right_eye ,25,(255,0,0), 2, cv2.LINE_AA)
                cv2.circle(image,left_eye ,25,(0,255,0), 2, cv2.LINE_AA)
                cv2.circle(image,nose_tip ,25,(0,255,255), 2, cv2.LINE_AA)
                
   
                
        # Flip the image horizontally for a selfie-view display.
        # cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5,fy=0.5))
        cv2.imshow('MediaPipe Face Detection', image)

        if cv2.waitKey(1) == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

NameError: name 'overlay' is not defined

In [ ]:
import cv2
import mediapipe as mp

def overay(image, x, y, w, h, overay_image): # 대상이미지 (3채널), x,y , width, height, 덮어씌움 이미지 
    alpha = overay_image[:, :, 3] # BGRA
    mask_image = alpha / 255 # 0 ~ 255로 나누면 0~1 사이값 (1 불투명 : 0 완전투명 )
    # (255,255) -> (1, 1)
    # (255, x) -> (1, 0.3)
    # 1 - mask_image ?
    # (0, 0)
    # (0, 1)
    
    for c in range(0,3): # channel BGR 
        image[y-h: y+h, x-w: x+w, c] =(overay_image[:, :, c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1 - mask_image))
        

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈을 사용 
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 Drawing_Utils 모듈을 사용 

# 영상 소스 선택 0 이면 WebCam임 
cap = cv2.VideoCapture(0) # Webcam 
# cap = cv2.VideoCapture('face_video.mp4') # Video

# 이미지 불러오기
image_right_eye = cv2.imread('right_eye.png')
image_left_eye = cv2.imread('left_eye.png')
image_nose_tip = cv2.imread('nose.png')

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.9) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 디텍션 된 얼굴에 무언가를 그려줌
        if results.detections:
            # 6개 특징 : 오른쪽눈, 왼쪽눈, 코끝부분, 입중심, 오른쪽 귀, 왼쪽귀 
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)
                # print(detection)
                
                # 특정 위치 가져오기 
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]
                nose_tip = keypoints[2]
                
                h, w,_ = image.shape
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100)
                left_eye = (int(left_eye.x * w) + 20, int(left_eye.y * h) - 100)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # 각 특징에 이미지 그리기 
                
                image[right_eye[1] - 50 : right_eye[1] + 50, right_eye[0] - 50 : right_eye[0] + 50] = image_right_eye
                image[left_eye[1] - 50 : left_eye[1] + 50, left_eye[0] - 50 : left_eye[0] + 50] = image_left_eye
                image[nose_tip[1] - 50 : nose_tip[1] + 50, nose_tip[0] - 150 : nose_tip[0] + 150] = image_nose_tip
                
                
        # Flip the image horizontally for a selfie-view display.
        # cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5,fy=0.5))
        cv2.imshow('MediaPipe Face Detection', image)

        if cv2.waitKey(1) == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

# 캐릭터 이미지 씌우기

In [18]:
import cv2
import mediapipe as mp

def overlay(image, x, y, w, h, overlay_image): # 대상이미지 (3채널), x,y , width, height, 덮어씌움 이미지 
    alpha = overlay_image[:, :, 3] # BGRA
    mask_image = alpha / 255 # 0 ~ 255로 나누면 0~1 사이값 (1 불투명 : 0 완전투명 )
    # (255,255) -> (1, 1)
    # (255, x) -> (1, 0.3)
    # 1 - mask_image ?
    # (0, 0)
    # (0, 1)
    
    for c in range(0, 3): # channel BGR 
        image[y-h: y+h, x-w: x+w, c] =(overay_image[:, :, c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1 - mask_image))
        

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈을 사용 
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 Drawing_Utils 모듈을 사용 

# 영상 소스 선택 0 이면 WebCam임 
# cap = cv2.VideoCapture(0) # Webcam 
cap = cv2.VideoCapture('face_video.mp4') # Video

# 이미지 불러오기
image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED)
image_left_eye = cv2.imread('left_eye.png', cv2.IMREAD_UNCHANGED)
image_nose_tip = cv2.imread('nose.png', cv2.IMREAD_UNCHANGED)

with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.9) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 디텍션 된 얼굴에 무언가를 그려줌
        if results.detections:
            # 6개 특징 : 오른쪽눈, 왼쪽눈, 코끝부분, 입중심, 오른쪽 귀, 왼쪽귀 
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)
                # print(detection)
                
                # 특정 위치 가져오기 
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0]
                left_eye = keypoints[1]
                nose_tip = keypoints[2]
                
                h, w, _ = image.shape
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100)
                left_eye = (int(left_eye.x * w) + 20, int(left_eye.y * h) - 100)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # 각 특징에 이미지 그리기 
                overlay(image, *right_eye, 50 ,50, image_right_eye)
                overlay(image, *left_eye, 50 ,50, image_left_eye)
                overlay(image, *nose_tip, 150 ,50, image_nose_tip) 
                
        # Flip the image horizontally for a selfie-view display.
        # cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5,fy=0.5))
        cv2.imshow('MediaPipe Face Detection', image)

        if cv2.waitKey(1) == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

IndexError: index 3 is out of bounds for axis 2 with size 3

In [17]:
image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED)
image_right_eye.shape

(100, 100, 3)

In [19]:
import cv2
import mediapipe as mp

def overlay(image, x, y, w, h, overlay_image): # 대상 이미지 (3채널), x, y 좌표, width, height, 덮어씌울 이미지 (4채널)
    alpha = overlay_image[:, :, 3] # BGRA
    mask_image = alpha / 255 # 0 ~ 255 -> 255 로 나누면 0 ~ 1 사이의 값 (1: 불투명, 0: 완전)
    # (255, 255)  ->  (1, 1)
    # (255, 0)        (1, 0)
    
    # 1 - mask_image ?
    # (0, 0)
    # (0, 1)
    
    for c in range(0, 3): # channel BGR
        image[y-h:y+h, x-w:x+w, c] = (overlay_image[:, :, c] * mask_image) + (image[y-h:y+h, x-w:x+w, c] * (1 - mask_image))

# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수 정의
mp_face_detection = mp.solutions.face_detection # 얼굴 검출을 위한 face_detection 모듈을 사용
mp_drawing = mp.solutions.drawing_utils # 얼굴의 특징을 그리기 위한 drawing_utils 모듈을 사용

# 동영상 파일 열기
cap = cv2.VideoCapture('face_video.mp4')

# 이미지 불러오기
image_right_eye = cv2.imread('right_eye.png', cv2.IMREAD_UNCHANGED) # 100 x 100
image_left_eye = cv2.imread('left_eye.png', cv2.IMREAD_UNCHANGED) # 100 x 100
image_nose = cv2.imread('nose.png', cv2.IMREAD_UNCHANGED) # 300 x 100 (가로, 세로)

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # Draw the face detection annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:
            # 6개 특징 : 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀 (귀구슬점, 이주)
            for detection in results.detections:
                # mp_drawing.draw_detection(image, detection)
                # print(detection)
                
                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye = keypoints[0] # 오른쪽 눈
                left_eye = keypoints[1] # 왼쪽 눈
                nose_tip = keypoints[2] # 코 끝부분
                
                h, w, _ = image.shape # height, width, channel : 이미지로부터 세로, 가로 크기 가져옴
                right_eye = (int(right_eye.x * w) - 20, int(right_eye.y * h) - 100) # 이미지 내에서 실제 좌표 (x, y)
                left_eye = (int(left_eye.x * w) + 20, int(left_eye.y * h) - 100)
                nose_tip = (int(nose_tip.x * w), int(nose_tip.y * h))
                
                # 양 눈에 동그라미 그리기
                # cv2.circle(image, right_eye, 50, (255, 0, 0), 10, cv2.LINE_AA) # 파란색
                # cv2.circle(image, left_eye, 50, (0, 255, 0), 10, cv2.LINE_AA) # 초록색                
                # 코에 동그라미 그리기
                # cv2.circle(image, nose_tip, 75, (0, 255, 255), 10, cv2.LINE_AA) # 노란색
                
                # 각 특징에다가 이미지 그리기                
                #image[right_eye[1]-50:right_eye[1]+50, right_eye[0]-50:right_eye[0]+50] = image_right_eye
                #image[left_eye[1]-50:left_eye[1]+50, left_eye[0]-50:left_eye[0]+50] = image_left_eye
                #image[nose_tip[1]-50:nose_tip[1]+50, nose_tip[0]-150:nose_tip[0]+150] = image_nose
                
                # image, x, y, w, h, overlay_image
                overlay(image, *right_eye, 50, 50, image_right_eye)
                overlay(image, *left_eye, 50, 50, image_left_eye)
                overlay(image, *nose_tip, 150, 50, image_nose)
                
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.5, fy=0.5))
        
        if cv2.waitKey(1) == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()

IndexError: index 3 is out of bounds for axis 2 with size 3